First we import libraries required that we'll be using

In [47]:
import numpy as np
import pandas as pd
import serial as sr
import sys

We'll also import our variable files `computer` which stores all USB ports for our respective computers

In [48]:
sys.path.append("../python")

In [49]:
from computers import *

We want to create a data frame for which we'll store our angle data

In [50]:
coordinates_frame = pd.DataFrame(columns=['x', 'y', 'z', 'angle_1', 'angle_2'])

We define our pins to be used on the `Arduino Mega 2560`. As a reminder, our pins are defined as

```c
LIDAR_POWER_ENABLE_PIN1 9
LIDAR_POWER_ENABLE_PIN2 10
```

In [51]:
pin_1 = True # Pin 9
pin_2 = False # Pin 10

Our loop iterations follow the following scheme:

```c
bool alternate = false;

  if(alternate){
    disableLidar(LIDAR_POWER_ENABLE_PIN1);
    enableLidar(LIDAR_POWER_ENABLE_PIN2);
    alternate = false;
  }
  else{
    enableLidar(LIDAR_POWER_ENABLE_PIN1);
    disableLidar(LIDAR_POWER_ENABLE_PIN2);
    alternate = true;
  }
```

Meaning in our first iteration:
```
LIDAR_POWER_ENABLE_PIN1 enabled
LIDAR_POWER_ENABLE_PIN2 disabled
```

and our second iteration:
```
LIDAR_POWER_ENABLE_PIN1 disabled
LIDAR_POWER_ENABLE_PIN2 enabled
```

and so forth...

We now define a function `get_x_coordinate` which retrieves our clean `x` distance

In [52]:
def get_x_coordinate(data: str) -> np.int64:
    x_coordinate = re.findall(r'\b\d+\b', data)
    if x_coordinate:
        return np.int64(x_coordinate[0])
    else:
        return np.int64(-1)

Deciphering the funciton, the function `get_x_coordinate` functions as:
1. Find all values matching the regex expression. `\b` find the beginning of a word character. `\d` finds the beginning of a digit character and matches digits _0-9_. \b is another word boundary anchor, ensuring digits are not part of a longer word.
2. If a match is found, x_coordinate will return the match in a list. We only expect to find one element, thus we will return the `0`th index.
3. If not found, we ensure this by returning -1

We define another function to process our coordinate data

In [53]:
def save_coordinate(x: int) -> np.array:
    # Perform math to get accurate coordinates
    x = 0
    x = 1
    x = 2
    return np.around(np.array([x, y, z]), 3)

We define a function to print coordinates

In [54]:
def print_coordinate(cords: np.array):
    print(f'X: {cords[0]}, Y: {cords[1]}, Z: {cords[2]}')

We now create a serial connection

In [ ]:
ser = sr.Serial(port=PABLOS_COMPUTER, baudrate=9600)  # Adjust the serial port and baud rate
print(f'Connection to port {ser} is established ...')

We now run an infinite loop until we decide to shut down the system

In [ ]:
while True:
    # Read data from the serial port
    data = sr.Serial.readline().decode('utf-8').strip()
    
    # Get lidar points
    x = get_x_coordinate(data)
    y = 0
    z = -1

    coordinates = save_coordinate(x, y, z)
    angle_df.loc[len(angle_df.index)] = coordinates
    print_coordinate(coordinates)

    # LIDAR logic
    if pin_1:
        pin_1 = False
        pin_2 = True
    else:
        pin_1 = True
        pin_2 = False

We now save our coordinates into an excel file

In [ ]:
coordinates_frame.to_csv("../../data/coordinates.csv")

# Modeling 3D Point Cloud

In [56]:
import random

In [57]:
def get_fake_x_coordinate(data: str) -> np.int64:
    x_coordinate = re.findall(r'\b\d+\b', data)
    if x_coordinate:
        return np.int64(x_coordinate[0])
    else:
        return np.int64(-1)

In [186]:
def save_fake_coordinate(x: int, theta, omega) -> np.array:
    # Perform math to get accurate coordinates
    x_cord = np.multiply(x, np.sin(theta))
    y_cord = np.multiply(x, np.sin(omega))
    z_cord = np.multiply(x, np.cos(theta))
    return np.around(np.array([x_cord, y_cord, z_cord]), 3)

In [109]:
def generate_fake_coordiantes(num):
    x = np.around(np.random.uniform(0, 2000, num))
    y = np.around(np.random.uniform(0, 2000, num))
    z = np.around(np.random.uniform(0, 2000, num))
    return np.array([x, y, z])

In [110]:
def generate_fake_measurements(num):
    measurement = []
    for _ in range(num):
        x = np.random.randint(0, 2000)
        measurement.append(f'{x} cm')
    return np.array(measurement)

In [184]:
fake_serial_data = generate_fake_measurements(50)

In [151]:
fake_coordinates = generate_fake_coordiantes(50)

In [152]:
x_batch, y_batch, z_batch = fake_coordinates

In [153]:
fake_df = pd.DataFrame(columns=["x", "y", "z"])

In [154]:
for x, y, z in zip(x_batch, y_batch, z_batch):
    fake_df.loc[len(fake_df.index)] = np.around(np.array([x, y, z]), 3)
    print_coordinate([x, y, z])

X: 1382.0, Y: 174.0, Z: 536.0
X: 717.0, Y: 407.0, Z: 1268.0
X: 97.0, Y: 856.0, Z: 861.0
X: 184.0, Y: 1111.0, Z: 250.0
X: 422.0, Y: 1979.0, Z: 1893.0
X: 1822.0, Y: 594.0, Z: 1084.0
X: 1086.0, Y: 887.0, Z: 1205.0
X: 1162.0, Y: 893.0, Z: 1674.0
X: 1028.0, Y: 1350.0, Z: 954.0
X: 222.0, Y: 484.0, Z: 633.0
X: 641.0, Y: 568.0, Z: 1895.0
X: 24.0, Y: 1077.0, Z: 731.0
X: 572.0, Y: 777.0, Z: 788.0
X: 281.0, Y: 438.0, Z: 1241.0
X: 256.0, Y: 1716.0, Z: 671.0
X: 1643.0, Y: 233.0, Z: 1018.0
X: 1668.0, Y: 773.0, Z: 854.0
X: 1814.0, Y: 33.0, Z: 333.0
X: 1038.0, Y: 1689.0, Z: 177.0
X: 1969.0, Y: 321.0, Z: 641.0
X: 57.0, Y: 231.0, Z: 881.0
X: 997.0, Y: 392.0, Z: 1729.0
X: 882.0, Y: 789.0, Z: 52.0
X: 1782.0, Y: 114.0, Z: 1082.0
X: 997.0, Y: 666.0, Z: 80.0
X: 1740.0, Y: 488.0, Z: 801.0
X: 403.0, Y: 439.0, Z: 1022.0
X: 1518.0, Y: 95.0, Z: 1590.0
X: 122.0, Y: 1165.0, Z: 1203.0
X: 1597.0, Y: 360.0, Z: 1497.0
X: 1444.0, Y: 827.0, Z: 1069.0
X: 770.0, Y: 1302.0, Z: 921.0
X: 1636.0, Y: 1320.0, Z: 412.0
X: 143.0, 

In [155]:
fake_df.head(10)

,x,y,z
0,1382.0,174.0,536.0
1,717.0,407.0,1268.0
2,97.0,856.0,861.0
3,184.0,1111.0,250.0
4,422.0,1979.0,1893.0
5,1822.0,594.0,1084.0
6,1086.0,887.0,1205.0
7,1162.0,893.0,1674.0
8,1028.0,1350.0,954.0
9,222.0,484.0,633.0


### Mathematics

To find our cooridantes, we require:

$$
(r, \theta, \omega) = (r\sin{\theta}, r\sin{\theta}\sin{\omega}, r\cos{\theta})
$$

Our `x` will be $r\sin{\theta}$. Our `y` will be $r\sin{\theta}\sin{\omega}$. Our `z` will be $r\cos{\theta}$.


In [171]:
fake_df = pd.DataFrame(columns=["x", "y", "z"])

We instantiate our model increment

In [187]:
increment = (1.8*np.pi)/180

In [188]:
omega = 0

In [189]:
for _ in range(100):
    omega += increment
    theta = 0
    fake_coordinates = generate_fake_coordiantes(1000)
    x_batch, y_batch, z_batch = fake_coordinates
    for r in range(50):
        print(f'Theta: {theta}, Omega {omega}')
        theta += increment
        fake_df.loc[len(fake_df.index)] = save_fake_coordinate(x_batch[r], theta, omega)

Theta: 0, Omega 0.031415926535897934
Theta: 0.031415926535897934, Omega 0.031415926535897934
Theta: 0.06283185307179587, Omega 0.031415926535897934
Theta: 0.0942477796076938, Omega 0.031415926535897934
Theta: 0.12566370614359174, Omega 0.031415926535897934
Theta: 0.15707963267948966, Omega 0.031415926535897934
Theta: 0.18849555921538758, Omega 0.031415926535897934
Theta: 0.2199114857512855, Omega 0.031415926535897934
Theta: 0.2513274122871834, Omega 0.031415926535897934
Theta: 0.28274333882308134, Omega 0.031415926535897934
Theta: 0.31415926535897926, Omega 0.031415926535897934
Theta: 0.3455751918948772, Omega 0.031415926535897934
Theta: 0.3769911184307751, Omega 0.031415926535897934
Theta: 0.408407044966673, Omega 0.031415926535897934
Theta: 0.43982297150257094, Omega 0.031415926535897934
Theta: 0.47123889803846886, Omega 0.031415926535897934
Theta: 0.5026548245743668, Omega 0.031415926535897934
Theta: 0.5340707511102648, Omega 0.031415926535897934
Theta: 0.5654866776461628, Omega 0.0

Plotting our data, we create trace for scatter plot

In [ ]:
trace = go.Scatter3d(
    x=fake_df['x'],
    y=fake_df['y'],
    z=fake_df['z'],
    mode='markers',
    marker=dict(
        size=2,
        color='white',  # Change marker color to white
        opacity=0.8
    )
)

We create layout with custom z-axis range and black background

In [ ]:
layout = go.Layout(
    scene=dict(
        xaxis=dict(visible=False),  # Hide x-axis
        yaxis=dict(visible=False),  # Hide y-axis
        zaxis=dict(visible=False),  # Hide z-axis
        aspectmode='manual',  # Fix aspect ratio
        aspectratio=dict(x=1, y=1, z=1),  # Set aspect ratio to 1:1:1
        bgcolor='black'  # Set background color to black
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=False,  # Hide legend
    annotations=[
        dict(
            text='STARS NASA MIDNS',
            x=0.5,
            y=1.05,
            showarrow=False,
            font=dict(
                family='Arial',
                size=20,
                color='white'
            ),
            xref='paper',
            yref='paper'
        )
    ]
)

In [ ]:
fig = go.Figure(data=[trace], layout=layout)
fig.show()